<h2>Testing the model's performance<h2>

In [63]:
import os
import cv2
import math
import copy
import time
import random
from matplotlib import pyplot as plt

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision.transforms import v2
from torchvision import datasets

from torchvision.transforms import v2

In [64]:
train_mode = False
eval_mode = True

In [65]:
torch.manual_seed(41)
torch.cuda.manual_seed(41)

In [66]:
class ScanModel(nn.Module):
    def __init__(self,):
        super().__init__()
        self.conv1 = nn.Conv2d(1,64,stride=1,kernel_size=3, bias=False)
        self.conv2 = nn.Conv2d(64,64,stride=1,kernel_size=3, bias=False)
        self.conv3 = nn.Conv2d(64,128,stride=1,kernel_size=3, bias=False)
        self.conv4 = nn.Conv2d(128,256,stride=1,kernel_size=3, bias=False)
        self.conv5 = nn.Conv2d(256,512, stride=1,kernel_size=3, bias=False)

        self.conv6 = nn.Conv2d(512,512, stride=1,kernel_size=3)#tuto layer nevim jestli pouzit

        self.dropout = nn.Dropout(0.22)
       
        self.bn1 = nn.BatchNorm2d(64)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
        self.bn4 = nn.BatchNorm2d(256)
        self.bn5 = nn.BatchNorm2d(512)
        self.bn6 = nn.BatchNorm2d(512)
         
        self.fc1 = nn.Linear(512*5*5, 128)
        self.fc2 = nn.Linear(128, 256)
        self.fc3 = nn.Linear(256,3)

    def forward(self, x):       #toto se da udelat chytreji kdyz vytvorite funkci ktera vezme x a aplikuje ty 3 veci - conv,relu a dropout
        x = F.relu(self.conv1(x))
        x = self.bn1(x)
        x = self.dropout(x)
        x = F.max_pool2d(x,2,2)

        x = F.relu(self.conv2(x))
        x = self.bn2(x)
        x = self.dropout(x)
        x = F.max_pool2d(x,2,2)

        x = F.relu(self.conv3(x))
        x = self.bn3(x)
        x = self.dropout(x)
        x = F.max_pool2d(x,2,2)

        x = F.relu(self.conv4(x))
        x = self.bn4(x)
        x = self.dropout(x)
        x = F.max_pool2d(x,2,2)


        x = F.relu(self.conv5(x))
        x = self.bn5(x)
        x = self.dropout(x)
        x = F.max_pool2d(x,2,2)
        
        x = x.view(-1,512*5*5)
        
        x = F.relu(self.fc1(x))
        x=self.dropout(x)
        x = F.relu(self.fc2(x))
        x= self.dropout(x)
        x = F.log_softmax(self.fc3(x), dim=1)
                   
        return x


In [67]:
transforms = v2.Compose([
    v2.Grayscale(num_output_channels=1),
    v2.Resize((244,244)),
    v2.ToTensor(),
    v2.Normalize(mean=[0.49], std=[0.225]),
])

c:\Users\lexsu\anaconda3\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:43: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


In [68]:
test_dataset = datasets.ImageFolder(root="Data/test", # target folder of images
                                  transform=transforms, # transforms to perform on data (images)
                                  target_transform=None) # transforms to perform on labels (if necessary)

In [69]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [70]:
model = ScanModel()
model.load_state_dict(torch.load("CTscan_class_model_8epochs.pt"))
model = model.to(device)

In [71]:
criterion = nn.CrossEntropyLoss()

In [72]:
test_data = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [73]:
test_correct = 0
test_loss = 0
model.eval()

ScanModel(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (conv5): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (conv6): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1))
  (dropout): Dropout(p=0.22, inplace=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn5): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn6): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, t

In [74]:
if eval_mode:
   with torch.no_grad():
         for batch,(X_test,y_test) in enumerate(test_data):
            X_test,y_test = X_test.to(device).float(), y_test.to(device)
            y_val = model(X_test)
            predicted = torch.max(y_val.detach(),1)[1] #detach udela to ze dostaneme cisty tensor z y_val a torch.max returne tuple dvou tensorů
            # The first tensor contains the maximum values. The second tensor contains the indices where the maximum values occur.
            for i in range(len(predicted)):
               if predicted[i] == y_test[i]:
                  test_correct += 1
               else: print(f"Incorrect: {predicted[i]}, should be {y_test[i]}")
               
            #test_correct += (predicted == y_test).sum() # ten sum udela to ze to True nebo False pretvori v 1 or 0
            if batch%3 == 0:
               print(f"Batch {batch} done.")
            loss = criterion(y_val,y_test)
            test_loss += loss

Incorrect: 2, should be 1
Incorrect: 2, should be 1
Incorrect: 2, should be 1
Incorrect: 2, should be 1
Incorrect: 2, should be 1
Batch 0 done.
Incorrect: 1, should be 2
Incorrect: 2, should be 1
Incorrect: 2, should be 1
Incorrect: 2, should be 1
Incorrect: 2, should be 1
Incorrect: 2, should be 1
Incorrect: 2, should be 1
Incorrect: 1, should be 2
Incorrect: 2, should be 1
Incorrect: 2, should be 1
Incorrect: 2, should be 1
Batch 3 done.
Incorrect: 2, should be 1
Incorrect: 2, should be 1
Incorrect: 2, should be 1
Incorrect: 2, should be 1
Incorrect: 2, should be 1
Incorrect: 2, should be 1
Incorrect: 2, should be 1
Incorrect: 2, should be 1
Incorrect: 1, should be 2
Incorrect: 2, should be 1
Incorrect: 2, should be 1
Incorrect: 2, should be 1
Incorrect: 2, should be 1
Incorrect: 2, should be 0
Incorrect: 2, should be 1
Incorrect: 2, should be 1
Batch 6 done.
Incorrect: 2, should be 1
Incorrect: 2, should be 1
Incorrect: 2, should be 1
Incorrect: 2, should be 1
Incorrect: 2, should b

In [75]:
if eval_mode:
    lst1 = os.listdir('Data/test/COVID19')
    lst2 = os.listdir('Data/test/NORMAL')
    lst3 = os.listdir('Data/test/PNEUMONIA') # your directory path
    lst = lst1 + lst2 + lst3
    num_samples = len(lst)
    print(f"Loss: {test_loss} \nAccuracy: {test_correct/num_samples}")

Loss: 7.329805850982666 
Accuracy: 0.9262422360248447


In [76]:
num_samples

1288

In [77]:
test_correct

1193

<h5>accuracy after 5 epochs of training: ~ 83.2%<h5>
<h5>accuracy after 6 epochs of training: ~ 92.7%<h5>
<h5>accuracy after 8 epochs of training: ~ 92.6%<h5>
We could improve this by augumenting our data (mainly for covid samples) and thus balancing the dataset

<h2>Continuing the training<h2>

In [78]:
if not train_mode:
    raise SyntaxError

SyntaxError: None (<string>)

In [ ]:
train_dataset = datasets.ImageFolder(root="Data/train", transform=transforms)
train_data = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [ ]:
EPOCHS = 2

In [ ]:
optimizer = torch.optim.Adam(params=model.parameters(),lr = 0.001)

In [ ]:
train_losses = []
train_correct = []

start_time = time.time()

for epoch in range(EPOCHS):

    epoch_train_correct = 0
    model.train()

    for batch, (X_train,y_train) in enumerate(train_data):
      X_train,y_train = X_train.to(device).float(), y_train.to(device)
      y_pred = model.forward(X_train)
      loss = criterion(y_pred,y_train)

      predicted = torch.max(y_pred.detach(),1)[1]   # .data oddelí gradient atd od samostatných dat a dá nám jen data, ta jednice v tom maxu znamena dimenze na ktere hleda max(cols nebo rows asi)
      batch_correct = (predicted == y_train).sum()
      epoch_train_correct += batch_correct

      optimizer.zero_grad()
      loss.backward() #vypocita gradient pro nase weights atd
      optimizer.step() #updatne weights s gradientem

      if batch%10 == 0:
        print(f"Epoch: {epoch+1}   Batch: {batch+1}   Loss: {loss.item()}")




    train_losses.append((f"Trl{epoch+1}",loss))
    train_correct.append((f"Trc{epoch+1}",epoch_train_correct))


current_time = time.time()
total_time = current_time - start_time
print(f"Training took {total_time/60} minutes.")

Epoch: 1   Batch: 1   Loss: 0.12195540964603424
Epoch: 1   Batch: 11   Loss: 0.06627944856882095
Epoch: 1   Batch: 21   Loss: 0.15441067516803741
Epoch: 1   Batch: 31   Loss: 0.20309525728225708
Epoch: 1   Batch: 41   Loss: 0.05238477140665054
Epoch: 1   Batch: 51   Loss: 0.10936272144317627
Epoch: 1   Batch: 61   Loss: 0.061345960944890976
Epoch: 1   Batch: 71   Loss: 0.059761181473731995
Epoch: 1   Batch: 81   Loss: 0.1293635070323944
Epoch: 2   Batch: 1   Loss: 0.10435722768306732
Epoch: 2   Batch: 11   Loss: 0.1989828497171402
Epoch: 2   Batch: 21   Loss: 0.12737734615802765
Epoch: 2   Batch: 31   Loss: 0.1885148286819458
Epoch: 2   Batch: 41   Loss: 0.04979225620627403
Epoch: 2   Batch: 51   Loss: 0.1975577175617218
Epoch: 2   Batch: 61   Loss: 0.13188600540161133
Epoch: 2   Batch: 71   Loss: 0.07148662954568863
Epoch: 2   Batch: 81   Loss: 0.009582582861185074
Training took 7.511333040396372 minutes.


In [ ]:
torch.save(model.state_dict(), 'CTscan_class_model_8epochs.pt')